
# Introduction

[Scrapy](https://scrapy.org/) est un framework permettant de crawler des
sites web et d'en extraire les données de façon structurée.

## Installation

[Scrapy](https://scrapy.org/) ne fait pas partie de la distribution par
défaut de Python et doit être installé manuellement. Ici, le package est
déjà installé grâce à Pipenv.

Si vous avez besoin de l'installer dans un autre cadre.

-   Avec **Pipenv** : `pipenv install scrapy==2.11.2`

**Vérifiez que vous avez bien activé votre environnement virtuel Python !!**

In [1]:
!where python

c:\Users\nolan\.virtualenvs\DataEngineerTools-r2yFN37n\Scripts\python.exe
C:\Users\nolan\AppData\Local\Programs\Python\Python312\python.exe
C:\Users\nolan\AppData\Local\Microsoft\WindowsApps\python.exe


In [2]:
import scrapy

## Architecture

[Scrapy](https://scrapy.org/) est un framework comportant plusieurs
composants.

<img src="images/architecture.png" alt="image" class="align-center" />

L'ensemble du processus est contrôlé par l'**engine** (les termes anglo
saxons ont été retenus pour un meilleur référencement dans la
[documentation officielle](https://docs.scrapy.org/en/latest/)).

Le framework est articulé avec plusieurs composants qui gèrent chacun un
rôle différent. Nous allons les détailler.

-   Les **Spiders** : permettent de naviguer sur un site et de
    référencer les règles d'extraction de la donnée.
-   Les **Pipelines** : font le lien entre la donnée brute et des objets
    structurés
-   Les **Middlewares** : permettent d'effectuer des transformations sur
    les objets ou sur les requêtes exécutées par l'engine.
-   Le **Scheduler** : gère l'ordre et le timing des requêtes
    effectuées.

## Fonctionnement

[Scrapy](https://scrapy.org/) est entièrement organisé autour d'un
composant central : l'*engine*.

Le rôle de l'*engine* est de contrôler le flux de données entre les
différents composants du système.

1.  En particulier, il est chargé de récupérer les *requests* définies
    dans les *spiders*
2.  Ces *requests* sont ensuite fournies au *scheduler* qui se charge de
    leur ordonnancement
3.  Les *requests* sont présentées selon cet ordonnancement à
    l'*engine*...
4.  ... qui les transmet au *downloader*
5.  Le *downloader* effectue la *request* et transmet la *response* (le
    contenu de la page web) à l'*engine*...
6.  ... puis l'envoie au *spider* pour traitement
7.  Le *spider* génére des *items* qui sont transmis à l'*engine*
8.  Les *items* sont ensuite poussés dans un pipeline pour nettoyage,
    validation et stockage

Ce processus est répété jusqu'à épuisement des requêtes.

[Scrapy](https://scrapy.org/) est un [framework orienté
événements](https://en.wikipedia.org/wiki/Event-driven_architecture)
(basé sur [Twisted](https://twistedmatrix.com/)) permettant une
programmation asynchrone (non bloquante). C'est particulièrement
intéressant dans les opérations de scraping, puisque **le programme
n'attend pas le résultat d'une requête pour en lancer une autre**.

En effet, lorsque l'on sollicite une ressource (requête réseau, système
de fichier, etc.) en mode bloquant, l'exécution du programme est
suspendue le temps que la transaction avec la ressource se termine (par
exemple le temps qu'une page web soit complètement téléchargée).
L'intérêt de faire des appels non bloquants, c'est que l'on peut gérer
de multiples téléchargements en parallèle, et que le programme peut
continuer à tourner pendant ce temps.

# Un scraping élémentaire

Avant de rentrer dans les détails du framework, nous allons mettre en
oeuvre un premier script permettant de récupérer l'information présente
sur [la page web](http://evene.lefigaro.fr/citations/winston-churchill)
recensant les citations de [Sir Winston
Churchill](https://en.wikipedia.org/wiki/Winston_Churchill).

**Exercice**

Examiner le code source de cette page avec l'inspecteur de votre
navigateur. Identifier les éléments contenant l'information recherchée,
ici la chaîne de caractères contenant la citation proprement dite.

## Le code source

Le code utilisé est le suivant:

In [7]:
# %load citations_churchill_spider1.py
import scrapy


class ChurchillQuotesSpider(scrapy.Spider):
    name = "citations de Churchill"
    start_urls = ["https://www.transfermarkt.fr/", ]

    def parse(self, response):
        for cit in response.xpath('//div[@class="newsticker__box"]'):
            text_value = cit.xpath('a/text()').extract_first()
            yield {'text': text_value}

## Le fonctionnement

Le fonctionnement est le suivant:

-   On importe le module scrapy (2)
-   et on définit une sous classe de `scrapy.Spider` (4)
-   la variable `start_urls` contient la liste des pages à scraper (6)
-   On redéfinit la méthode $parse$ dont la signature est définie dans
    la classe mère (8)
-   L'objet
    [response](https://docs.scrapy.org/en/latest/topics/request-response.html#response-objects)
    représente la réponse à la requête HTTP (l'attribut $text$ permet
    d'accéder à son contenu). On recherche ensuite tous les containers
    `<div>` identifiés dans l'exercice précédent. Ici la page est
    particulièrement bien structurée et les citations disposent de leur
    propre container, identifié par l'attribut `class` de valeur
    `figsco__quote__text`. La sélection se fait par une expression
    [XPath](https://en.wikipedia.org/wiki/XPath), un langage de
    sélection de noeud dans un document XML (9). En langage naturel, la
    requête pourrait se formuler : "On recherche tous les containers
    `<div>` dont la valeur de l'attribut `class` est égal à
    `figsco__quote__text`".
-   Pour chaque résultat, on construit un dictionnaire dont la clé est
    `text` et la valeur le contenu du lien `<a>`. Ce résultat est fourni
    par un générateur ($yield$) (11).

On lance le scraping depuis un terminal (attention, ici on ne lance pas la cellule du jupyter notebook mais vraiment le script contenu dans le ifchier citations_churchill_spider1.py):

In [6]:
!scrapy runspider citations_churchill_spider1.py

2025-01-07 14:48:54 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2025-01-07 14:48:54 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.10.0, Python 3.12.5 (tags/v3.12.5:ff3bc82, Aug  6 2024, 20:45:27) [MSC v.1940 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Windows-11-10.0.26100-SP0
2025-01-07 14:48:54 [scrapy.addons] INFO: Enabled addons:
[]
2025-01-07 14:48:54 [py.warnings] WARNING: C:\Users\nolan\.virtualenvs\DataEngineerTools-r2yFN37n\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

On y trouve des informations sur les paramètres
utilisés:

```
2018-01-10 17:21:05 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2018-01-10 17:21:05 [scrapy.utils.log] INFO: Overridden settings: {'SPIDER_LOADER_WARN_ONLY': True}
```

les
[extensions](https://docs.scrapy.org/en/latest/topics/extensions.html)
...:

```
2018-01-10 17:21:05 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
'scrapy.extensions.telnet.TelnetConsole',
'scrapy.extensions.logstats.LogStats']
```

[Les composants middleware
downloader](https://docs.scrapy.org/en/latest/topics/downloader-middleware.html)
... :

```
2018-01-10 17:21:05 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
'scrapy.downloadermiddlewares.retry.RetryMiddleware',
'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware',
'scrapy.downloadermiddlewares.stats.DownloaderStats']
```

Idem pour [les composants middleware
spider](https://docs.scrapy.org/en/latest/topics/spider-middleware.html)
...:

```
2018-01-10 17:21:05 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
'scrapy.spidermiddlewares.referer.RefererMiddleware',
'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
'scrapy.spidermiddlewares.depth.DepthMiddleware']
```

Aucun
[pipeline](https://docs.scrapy.org/en/latest/topics/item-pipeline.html)
n'est activé :

```
2018-01-10 17:21:05 [scrapy.middleware] INFO: Enabled item pipelines:
[]
```

**Exercice**

Identifier la position des [composants middleware
downloader](https://docs.scrapy.org/en/latest/topics/downloader-middleware.html),
des [composants middleware
spider](https://docs.scrapy.org/en/latest/topics/spider-middleware.html)
et du
[pipeline](https://docs.scrapy.org/en/latest/topics/item-pipeline.html)
dans $l'architecture <Introduction>$

L'exécution du scraping proprement dit débute :
```
2018-01-10 17:21:05 [scrapy.core.engine] INFO: Spider opened
2018-01-10 17:21:05 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2018-01-10 17:21:05 [scrapy.extensions.telnet] DEBUG: Telnet console listening on 127.0.0.1:6023
```

La première URL est poussée par le scheduler:
```
2018-01-10 17:21:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://evene.lefigaro.fr/citations/winston-churchill> (referer: None)
```

## Les résultats

Les résultats sont fournis par le générateur défini dans la méthode
$parse$ dans un dictionnaire. Ils contiennent le texte des citations
dans la valeur de la clé `text` :
```
2018-01-10 17:21:05 [scrapy.core.scraper] DEBUG: Scraped from <200 http://evene.lefigaro.fr/citations/winston-churchill>
{'text': '“Le vice inhérent au capitalisme consiste en une répartition inégale des richesses. La vertu inhérente au socialisme consiste en une égale répartition de la misère.”'}
2018-01-10 17:21:05 [scrapy.core.scraper] DEBUG: Scraped from <200 http://evene.lefigaro.fr/citations/winston-churchill>
{'text': "Faire le bien, éviter le mal, c'est ça le paradis."}
```

## Les statistiques

Une fois le scraping effectué, quelques statistiques sont affichées sur
le terminal:
```
2018-01-10 17:21:05 [scrapy.core.engine] INFO: Closing spider (finished)
2018-01-10 17:21:05 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 242,
'downloader/request_count': 1,
'downloader/request_method_count/GET': 1,
'downloader/response_bytes': 17435,
'downloader/response_count': 1,
'downloader/response_status_count/200': 1,
'finish_reason': 'finished',
'finish_time': datetime.datetime(2018, 1, 10, 16, 21, 5, 858347),
'item_scraped_count': 16,
'log_count/DEBUG': 18,
'log_count/INFO': 7,
'response_received_count': 1,
'scheduler/dequeued': 1,
'scheduler/dequeued/memory': 1,
'scheduler/enqueued': 1,
'scheduler/enqueued/memory': 1,
'start_time': datetime.datetime(2018, 1, 10, 16, 21, 5, 645347)}
2018-01-10 17:21:05 [scrapy.core.engine] INFO: Spider closed (finished)
```

On observe notamment que notre code permet de récupérer la taille de la
page web (17435 bytes), le temps d'exécution à partir des valeurs
`finish_time` et `start_time`, le nombre d'items scrapés (16), etc...

**Exercice**

Les citations extraites sont elles toutes de [Sir Winston
Churchill](https://en.wikipedia.org/wiki/Winston_Churchill) ? Il sera
peut être nécessaire de modifier le sélecteur XPath. Nous verrons ça
lorsque il faudra récupérer les données relative à l'auteur.


## Modifier les données

Il est parfois nécessaire de faire un traitement sur les données
scrapées, pour ajouter ou retirer de l'information.

**Exercice**

Retirer les caractères `“` et `”` qui délimitent la citation. Ces
caractères sont identifiés en Unicode comme [LEFT DOUBLE QUOTATION
MARK](http://www.fileformat.info/info/unicode/char/201c/index.htm) et
[RIGHT DOUBLE QUOTATION
MARK](http://www.fileformat.info/info/unicode/char/201d/index.htm).

Attention, ici il faut bien modifier directement le script `citations_churchill_spider1.py` et lancer dans une cellule ou votre terminal la commande utilisée précédemment : `scrapy runspider citations_churchill_spider1.py`

## Plus de données

Il est souvent nécessaire de récupérer plusieurs informations relatives
à un même item. Dans cet exemple, il est judicieux d'associer à la
citation le nom de son auteur, en allant chercher cette information au
plus près du texte lui même.

**Exercice**

Examiner le code source de la page web et identifier la structuration de
la donnée associée à l'auteur. En déduire l'expression XPath permettant
de la récupérer. S'assurer que seules les citations de [Sir Winston
Churchill](https://en.wikipedia.org/wiki/Winston_Churchill) sont
extraites. Ajouter une clé `author` au dictionnaire retourné par le
$yield$ dont la valeur est précisément la chaîne de caractères contenant
l'auteur.

Un exemple de dictionnaire retourné:
```
{
    'text': "Si deux hommes ont toujours la même opinion, l'un d'eux est de trop.", 
    'author': 'Winston Churchill'
}
```

si vous voulez stocker les données scrapées dans un fichier vous pouvez grâce à :
```
scrapy runspider citations_churchill_spider1.py -o data/citation.json -t json
```


# Votre premier projet


Dans un premier temps vous devez créer un projet Scrapy avec la commande
:

In [6]:
!scrapy startproject monprojet

New Scrapy project 'monprojet', using template directory 'C:\Users\nolan\.virtualenvs\DataEngineerTools-r2yFN37n\Lib\site-packages\scrapy\templates\project', created in:
    C:\Users\nolan\OneDrive\Bureau\DataEngineerTools\2Scrapy\monprojet

You can start your first spider with:
    cd monprojet
    scrapy genspider example example.com


Cette commande va créer un dossier `monprojet` contenant les éléments
suivants correspondant au squelette :

```
newscrawler/
    scrapy.cfg            # Options de déploiement

    newscrawler/             # Le module Python contenant les informations
        __init__.py

        items.py          # Fichier contenant les items

        middlewares.py    # Fichier contenant les middlewares

        pipelines.py      # Fichier contenant les pipelines

        settings.py       # Fichier contenant les paramètres du projet

        spiders/          # Dossier contenant toutes les spiders
            __init__.py
```

**Attention, à partir de maintenant vous créerez et modifierez les fichiers python dans votre projet scrapy et non pas dans les cellules jupyter qui sont là juste pour l'exemple**

Pour la suite, un projet scrapy du nom de `newscrawler` est présent dans le repo pour vous guider, vous pouvez vous en inspirer pour implémenter vos propres spiders mais n'hésitez pas à explorer les méthodes que scrapy offre depuis la documentation.

# Votre première Spider

Une Spider est une classe Scrapy qui permet de mettre en place toute
l'architecture complexe vue dans l'introduction. Pour définir une
spider, il vous faut hériter de la classe $scrapy.Spider$. La seule
chose à faire est de définir la première requête à effectuer et comment
suivre les liens. La Spider s'arrêtera lorsqu'elle aura parcouru tous
les liens qu'on lui a demandé de suivre.

Pour créer une Spider on utilise la syntaxe:

In [7]:
!scrapy genspider <SPIDER_NAME> <DOMAIN_NAME>

< �tait inattendu.


Par exemple,

In [8]:
!cd newscrawler && scrapy genspider lemonde lemonde.fr

Spider 'lemonde' already exists in module:
  newscrawler.spiders.lemonde


Cette commande permet de créer une spider appelée `lemonde` pour scraper
le domaine `lemonde.fr`. Cela crée le fichier Python
`spiders/lemonde.py` suivant :

In [9]:
# %load newscrawler/newscrawler/spiders/lemonde.py
import scrapy


class LemondeSpider(scrapy.Spider):
    name = 'lemonde'
    allowed_domains = ['lemonde.fr']
    start_urls = ['http://lemonde.fr/']

    def parse(self, response):
        pass


Une bonne pratique pour commencer à développer une Spider est de passer
par l'interface Shell proposée par Scrapy. Elle permet de récupérer un
objet `Response` et de tester les méthodes de récupération des données.

### ATTENTION : Les commandes scrapy shell doivent être lancées dans un terminal et non dans des cellules jupyter

`$ scrapy shell 'http://lemonde.fr'`

Pour les utilisateurs de windows il vous faut mettre des doubles quotes
:
```
scrapy shell "http://lemonde.fr"
```

Scrapy lance un kernel Python

Grâce à cette interface, vous avez accès à plusieurs objets comme la
`Response`, la `Request`, la `Spider` par exemple. Vous pouvez aussi
exécuter `view(response)` pour afficher ce que Scrapy récupère dans un
navigateur.

Voici quelques exemples de commandes que vous pouvez faire dans le scrapy shell :
```
In [1]: response
Out[1]: <200 https://www.lemonde.fr/>

In [3]: request
Out[3]: <GET https://www.lemonde.fr/>

In [4]: type(request)
Out[4]: scrapy.http.request.Request

In [5]: spider
Out[5]: <LemondeSpider 'lemonde' at 0x1080fccc0>

In [6]: type(spider)
Out[6]: monprojet.spiders.lemonde.LeMondeSpider
```

Ici on voit que la Spider est une instance de LemondeSpider. Lorsqu'on
lance le $scrapy shell$ scrapy va chercher dans les spiders si une
correspond au lien passé en paramètre, si oui , il l'utilise sinon une
$DefaultSpider$ est instanciée.

## Vos premières requêtes

On peut commencer à regarder comment extraire les données de la page web
en utilisant le langage de requêtes proposé par Scrapy. Il existe deux
types de requêtes : les requêtes `css` et `xpath`. Les requêtes `xpath`
sont plus complexes mais plus puissantes que les requêtes `css`. Dans le
cadre de ce tutorial, nous allons uniquement aborder les requêtes `css`,
elles nous suffiront pour extraire les données dont nous avons besoin
(en interne, Scrapy transforme les requêtes `css`en requêtes `xpath`.

Que ce soit les requêtes `css` ou `xpath`, elles crééent des sélecteurs
de différents types. Quelques exemples :

Pour récupérer le titre d'une page :
```
In [1]: response.css('title')
Out[1]: [<Selector xpath='descendant-or-self::title' data='<title>Le Monde.fr - Actualités et Infos'>]
```

On récupère une liste de sélecteurs correspondant à la requête `css`
appelée. La requête précédente était unique, d'autres requêtes moins
restrictives permettent de récupérer plusieurs résultats. Par exemple
pour rechercher l'ensemble des liens présents sur la page, on va
rechercher les balises HTML `<a></a>`

```
In [5]: response.css("a")[0:10]
Out[5]:
[<Selector xpath='descendant-or-self::a' data='<a target="_blank" data-target="jelec-he'>,
<Selector xpath='descendant-or-self::a' data='<a href="/"> <div class="logo__lemonde l'>,
<Selector xpath='descendant-or-self::a' data='<a href="https://secure.lemonde.fr/sfuse'>,
<Selector xpath='descendant-or-self::a' data='<a href="https://abo.lemonde.fr/#xtor=CS'>,
<Selector xpath='descendant-or-self::a' data='<a href="/" class="Burger__right-arrow j'>,
<Selector xpath='descendant-or-self::a' data='<a href="/" class="Burger__right-arrow j'>,
<Selector xpath='descendant-or-self::a' data='<a href="#" class="js-dropdown Burger__r'>,
<Selector xpath='descendant-or-self::a' data='<a href="/mouvement-des-gilets-jaunes/" '>,
<Selector xpath='descendant-or-self::a' data='<a href="/carlos-ghosn/" data-suggestion'>,
<Selector xpath='descendant-or-self::a' data='<a href="/implant-files/" data-suggestio'>]
```

Pour récupérer le texte contenu dans les balises, on passe le paramètre
`<TAG>::text`. Par exemple :

```
In [6]: response.css("title::text")
Out[6]: [<Selector xpath='descendant-or-self::title/text()' data='Le Monde.fr - Actualités et Infos en Fra'>]
```

### Exercice

Comparer les résultats des deux requêtes `response.css('title')` et
`response.css('title::text')`.

Maintenant pour extraire les données des selecteurs on utilise l'une des
deux méthodes suivantes :
- `extract()` permet de récupérer une liste
des données extraites de tous les sélecteurs
- `extract_first()` permet
de récupérer une `String` provenant du premier sélecteur de la liste.


```
In [7]: response.css('title::text').extract_first()
Out[7]: 'Le Monde.fr - Actualités et Infos en France et dans le monde'
```

On peut récupérer un attribut d'une balise avec la syntaxe
`<TAG>::attr(<ATTRIBUTE_NAME>)` :

Par exemple, les liens sont contenus dans un attribut `href`:

```
In [9]: response.css('a::attr(href)')[0:10]
Out[9]:
[<Selector xpath='descendant-or-self::a/@href' data='https://journal.lemonde.fr'>,
<Selector xpath='descendant-or-self::a/@href' data='/'>,
<Selector xpath='descendant-or-self::a/@href' data='https://secure.lemonde.fr/sfuser/connexi'>,
<Selector xpath='descendant-or-self::a/@href' data='https://abo.lemonde.fr/#xtor=CS1-454[CTA'>,
<Selector xpath='descendant-or-self::a/@href' data='/'>,
<Selector xpath='descendant-or-self::a/@href' data='/'>,
<Selector xpath='descendant-or-self::a/@href' data='#'>,
<Selector xpath='descendant-or-self::a/@href' data='/mouvement-des-gilets-jaunes/'>,
<Selector xpath='descendant-or-self::a/@href' data='/carlos-ghosn/'>,
<Selector xpath='descendant-or-self::a/@href' data='/implant-files/'>]
```

Comme vu précédemment, si on veut récupérer la liste des liens de la page on applique la méthode $extract()$

```
In [11]: response.css('a::attr(href)').extract()[0:10]
Out[11]:
['https://journal.lemonde.fr',
'/',
'https://secure.lemonde.fr/sfuser/connexion',
'https://abo.lemonde.fr/#xtor=CS1-454[CTA_LMFR]-[HEADER]-5-[Home]',
'/',
'/',
'#',
'/mouvement-des-gilets-jaunes/',
'/carlos-ghosn/',
'/implant-files/']
```

Les liens dans une page HTML sont souvent codés de manière relative par
rapport à la page courante. La méthode de l'objet `Response` peut être
utilisée pour recréer l'url complet.

Un exemple sur le 4e élément :
```
In [14]: response.urljoin(response.css('a::attr(href)').extract()[8])
Out[14]: 'https://www.lemonde.fr/carlos-ghosn/'
```

alors que :
```
In [15]: response.css('a::attr(href)').extract()[8]
Out[15]: '/carlos-ghosn/'
```

### Exercice : 

Utiliser une liste compréhension pour transformer les 10
premiers liens relatifs récupérés par la méthode `extract()` en liens
absolus.

Le résultat doit ressembler à :
```
Out[23]: 
['https://journal.lemonde.fr',
'https://www.lemonde.fr/',
'https://secure.lemonde.fr/sfuser/connexion',
'https://abo.lemonde.fr/#xtor=CS1-454[CTA_LMFR]-[HEADER]-5-[Home]',
'https://www.lemonde.fr/',
'https://www.lemonde.fr/',
'https://www.lemonde.fr/',
'https://www.lemonde.fr/mouvement-des-gilets-jaunes/',
'https://www.lemonde.fr/carlos-ghosn/',
'https://www.lemonde.fr/implant-files/']
```

## Des requêtes plus complexes

On peut créer des requêtes plus complexes en utilisant à la fois la
structuration HTML du document mais également la couche de présentation
CSS. On utilise l'inspecteur de `Google Chrome` pour identifier le type
et l'identifiant de la balise contenant les informations.

Il y a au moins deux choses à savoir en `css` :

-   Les `.` représentent les classes
-   Les `#` représentent les id

On se propose de récupérer toutes les sous-catégories de news dans la
catégorie **Actualités**. On remarque en utilisant l'inspecteur
d'élement de Chrome que toutes les catégories sont rangées dans une
balise avec l'id $#nav-markup$ ensuite dans les classes $Nav__item$.

A partir de cette structure HTML on peut construire la requête suivante
pour récupérer la barre de navigation:

```
In [19]: response.css("#nav-markup")
Out[19]: [<Selector xpath="descendant-or-self::*[@id = 'nav-markup']" data='<ul id="nav-markup"> <li class="Nav__ite'>]
```


Ensuite pour récupérer les différentes catégories :

```
In [24]: response.css('#nav-markup').css('.Nav__item')
Out[24]:
[<Selector xpath="descendant-or-self::*[@id = 'nav-markup']/descendant-or-self::*/*[@class and contains(concat(' ', normalize-space(@class), ' '), ' Nav__item ')]" data='<li class="Nav__item js-burger-to-show N'>,
<Selector xpath="descendant-or-self::*[@id = 'nav-markup']/descendant-or-self::*/*[@class and contains(concat(' ', normalize-space(@class), ' '), ' Nav__item ')]" data='<li class="Nav__item Nav__item--home Nav'>,
<Selector xpath="descendant-or-self::*[@id = 'nav-markup']/descendant-or-self::*/*[@class and contains(concat(' ', normalize-space(@class), ' '), ' Nav__item ')]" data='<li class="Nav__item"> <a href="/" class'>,
<Selector xpath="descendant-or-self::*[@id = 'nav-markup']/descendant-or-self::*/*[@class and contains(concat(' ', normalize-space(@class), ' '), ' Nav__item ')]" data='<li class="Nav__item"> <a href="#" class'>,
<Selector xpath="descendant-or-self::*[@id = 'nav-markup']/descendant-or-self::*/*[@class and contains(concat(' ', normalize-space(@class), ' '), ' Nav__item ')]" data='<li class="Nav__item"> <a href="#" class'>,
<Selector xpath="descendant-or-self::*[@id = 'nav-markup']/descendant-or-self::*/*[@class and contains(concat(' ', normalize-space(@class), ' '), ' Nav__item ')]" data='<li class="Nav__item"> <a href="#" class'>,
<Selector xpath="descendant-or-self::*[@id = 'nav-markup']/descendant-or-self::*/*[@class and contains(concat(' ', normalize-space(@class), ' '), ' Nav__item ')]" data='<li class="Nav__item"> <a href="#" class'>,
<Selector xpath="descendant-or-self::*[@id = 'nav-markup']/descendant-or-self::*/*[@class and contains(concat(' ', normalize-space(@class), ' '), ' Nav__item ')]" data='<li class="Nav__item"> <a href="#" class'>,
<Selector xpath="descendant-or-self::*[@id = 'nav-markup']/descendant-or-self::*/*[@class and contains(concat(' ', normalize-space(@class), ' '), ' Nav__item ')]" data='<li class="Nav__item"> <a href="#" class'>,
<Selector xpath="descendant-or-self::*[@id = 'nav-markup']/descendant-or-self::*/*[@class and contains(concat(' ', normalize-space(@class), ' '), ' Nav__item ')]" data='<li class="Nav__item"> <a href="#" class'>,
<Selector xpath="descendant-or-self::*[@id = 'nav-markup']/descendant-or-self::*/*[@class and contains(concat(' ', normalize-space(@class), ' '), ' Nav__item ')]" data='<li class="Nav__item"> <a href="/recherc'>]
```

On veut maintenant retourner tous les liens présents dans cette
catégorie. On remarque qu'elle apparait à la 4eme position.
```
In [34]: response.css("#nav-markup .Nav__item")[3]
Out[34]: <Selector xpath="descendant-or-self::*[@id = 'nav-markup']/descendant-or-self::*/*[@class and contains(concat(' ', normalize-space(@class), ' '), ' Nav__item ')]" data='<li class="Nav__item"> <a href="#" class'>
```

Maintenant pour récupérer tous les liens, on peut chainer les requêtes.
On accède alors à toutes les balises $a$.

```
In [35]: response.css("#nav-markup .Nav__item")[3].css("a")
Out[35]:
[<Selector xpath='descendant-or-self::a' data='<a href="#" class="js-dropdown Burger__r'>,
<Selector xpath='descendant-or-self::a' data='<a href="/mouvement-des-gilets-jaunes/" '>,
<Selector xpath='descendant-or-self::a' data='<a href="/carlos-ghosn/" data-suggestion'>,
<Selector xpath='descendant-or-self::a' data='<a href="/implant-files/" data-suggestio'>,
<Selector xpath='descendant-or-self::a' data='<a href="/climat/" data-suggestion>Clima'>,
<Selector xpath='descendant-or-self::a' data='<a href="/affaire-khashoggi/" data-sugge'>,
<Selector xpath='descendant-or-self::a' data='<a href="/emmanuel-macron/" data-suggest'>,
<Selector xpath='descendant-or-self::a' data='<a href="/ukraine/" data-suggestion>Ukra'>,
<Selector xpath='descendant-or-self::a' data='<a href="/russie/" data-suggestion>Russi'>,
<Selector xpath='descendant-or-self::a' data='<a href="/referendum-sur-le-brexit/" dat'>,
<Selector xpath='descendant-or-self::a' data='<a href="/harcelement-sexuel/" data-sugg'>,
<Selector xpath='descendant-or-self::a' data='<a href="/actualite-en-continu/" data-su'>,
<Selector xpath='descendant-or-self::a' data='<a href="/international/">International<'>,
<Selector xpath='descendant-or-self::a' data='<a href="/politique/">Politique</a>'>,
<Selector xpath='descendant-or-self::a' data='<a href="/societe/">Société</a>'>,
<Selector xpath='descendant-or-self::a' data='<a href="/les-decodeurs/">Les Décodeurs<'>,
<Selector xpath='descendant-or-self::a' data='<a href="/sport/">Sport</a>'>,
<Selector xpath='descendant-or-self::a' data='<a href="/planete/">Planète</a>'>,
<Selector xpath='descendant-or-self::a' data='<a href="/sciences/">Sciences</a>'>,
<Selector xpath='descendant-or-self::a' data='<a href="/campus/">M Campus</a>'>,
<Selector xpath='descendant-or-self::a' data='<a href="/afrique/">Le Monde Afrique</a>'>,
<Selector xpath='descendant-or-self::a' data='<a href="/pixels/">Pixels</a>'>,
<Selector xpath='descendant-or-self::a' data='<a href="/actualite-medias/">Médias</a>'>,
<Selector xpath='descendant-or-self::a' data='<a href="/sante/">Santé</a>'>,
<Selector xpath='descendant-or-self::a' data='<a href="/big-browser/">Big Browser</a>'>,
<Selector xpath='descendant-or-self::a' data='<a href="/disparitions/">Disparitions</a'>]
```

Et pour récupérer les titres :
```
In [37]: response.css("#nav-markup .Nav__item")[3].css("a::text").extract()
Out[37]:
['Actualités',
 'Guerre au Proche-Orient',
 'Gouvernement Barnier',
 'Guerre en Ukraine',
 'Élection présidentielle américaine 2024',
 '7-Octobre, un an après',
 'Procès des viols de Mazan',
 'Donald Trump',
 'Climat',
 'Guadeloupe',
 'Toute l’actualité en continu',
 'International',
 'Politique',
 'Société',
 'Planète',
 'Climat',
 'Le Monde Afrique',
 'Les Décodeurs',
 'Sport',
 'Education',
 'M Campus',
 'Santé',
 'Intimités',
 'Sciences',
 'Pixels',
 'Disparitions',
 'Le Fil Good',
 'Podcasts',
 'Le Monde & Vous']
```

Le shell Scrapy permet de définir la structure des requêtes et de
s'assurer de la pertinence du résultat retourné. Pour automatiser le
processus, il faut intégrer cette syntaxe au code Python des modules de
spider définis dans la structure du projet.

## Intégration des requêtes

Le squelette de la classe `LeMondeSpider` généré lors de la création du
projet doit maintenant être enrichi. Par défaut 3 attributs et une
méthode `parse()` ont été créés :

-   `name` permet d'identifier sans ambiguïté la spider dans le code.
-   `allowed_domain` permet de filtrer les requêtes et forcer la spider
    à rester sur une liste de domaines.
-   `starts_urls` est la liste des urls d'où la spider va partir pour
    commencer son scraping.
-   `parse()` est une méthode héritée de la classe `scrapy.Spider`. Elle
    doit être redéfinie selon les requêtes que l'on doit effectuer et
    sera appelée sur l'ensemble des urls contenus dans la liste
    `starts_urls`.

`parse()` est une fonction `callback` qui sera appelée automatiquement
sur chaque objet `Response` retourné par la requête. Cette fonction est
appelée de manière asynchrone. Plusieurs requêtes peuvent ainsi être
lancées en parallèles sans bloquer le thread principal. L'objet
`Response` passé en paramètre est le même que celui mis à disposition
lors de l'exécution du Scrapy Shell.

In [9]:
def parse(self, response):
    title = response.css('title::text').extract_first()
    all_links = {
        name: response.urljoin(url) for name, url in zip(
            response.css("#nav-markup .Nav__item")[3].css("a::text").extract(),
            response.css("#nav-markup .Nav__item")[3].css("a::attr(href)").extract())
    }
    yield {
        "title": title,
        "all_links": all_links
    }

La fonction est un générateur (`yield`) et retourne un dictionnaire
composé de deux éléments :

-   Le titre de la page;
-   La liste des liens sortants sous forme de String.

Pour le moment cette spider ne parcourt que la page d'accueil, ce qui
n'est pas très productif.

## Votre premier scraper

Récupérer les données sur un ensemble de pages webs nécessite d'explorer
en profondeur la structure du site en suivant tout ou partie des liens
rencontrés.

La spider peut se `balader` sur un site assez efficacement. Il suffit de
lui indiquer comment faire. Il faut spécifier à Scrapy de générer une
requête vers une nouvelle page en construisant l'objet `Request`
correspondant. Ce nouvel objet `Request` est alors inséré dans le
scheduler de Scrapy. On peut évidemment générer plusieurs `Request`
simultanément, correspondant par exemple, à différents liens sur la page
courante. Ils sont insérés séquentiellement dans le scheduler.

Pour cela, on modifie la méthode `parse()` de façon à ce qu'elle retourne
un objet `Request` pour chaque nouveau lien rencontré. On associe
également à cet objet une fonction de callback qui déterminera la
manière dont cette nouvelle page doit être extraite.

Par exemple, pour que la spider continue dans les liens des différentes
régions (pour l'instant la fonction de callback ne fait rien) :

In [2]:
# %load newscrawler/newscrawler/spiders/lemonde_v2.py
import scrapy
from scrapy import Request


class LemondeSpider(scrapy.Spider):
    name = "lemondev2"
    allowed_domains = ["www.lemonde.fr"]
    start_urls = ['https://www.lemonde.fr']

    def parse(self, response):
        title = response.css('title::text').extract_first()
        all_links = {
            name: response.urljoin(url) for name, url in zip(
                response.css("#nav-markup .Nav__item")[3].css("a::text").extract(),
                response.css("#nav-markup .Nav__item")[3].css("a::attr(href)").extract())
        }
        yield {
            "title": title,
            "all_links": all_links
        }


Pour lancer la spider

In [3]:
!cd newscrawler && scrapy crawl lemondev2

2024-12-03 16:15:57 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: newscrawler)
2024-12-03 16:15:57 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.10.0, Python 3.12.5 (tags/v3.12.5:ff3bc82, Aug  6 2024, 20:45:27) [MSC v.1940 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Windows-11-10.0.26100-SP0
Traceback (most recent call last):
  File "C:\Users\nolan\.virtualenvs\DataEngineerTools-r2yFN37n\Lib\site-packages\scrapy\spiderloader.py", line 87, in load
    return self._spiders[spider_name]
           ~~~~~~~~~~~~~^^^^^^^^^^^^^
KeyError: 'lemondev2'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\nolan\.virtualenvs\DataEngineerTools-r2yFN37n\Scripts\scrapy.exe\__main__.py", line 7, in <modu

On veut ensuite *entrer* dans les liens des différentes sous-catégories
pour récupérer les articles. Pour cela, nous créons une méthode
`parse_category()` prend en argument un objet `Response` qui sera la
réponse correspondant aux liens des régions. On peut comme ceci
traverser un site en définissant des méthodes différentes en fonction du
type de contenu.

Si la structure du site est plus profonde, on peut empiler autant de
couches que souhaité.

Quand on arrive sur une page d'une sous-catégorie, on peut vouloir
récupérer tous les éléments de la page. Pour cela, on réutilise le
scrapy Shell pour commencer le développement de la nouvelle méthode
d'extraction.

Par exemple pour la page `https://www.lemonde.fr/international/` :

```
scrapy shell 'https://www.lemonde.fr/international/'
```

Le fil des articles est stocké dans une balise avec la classe
`class=river`.

```
In [3]: response.css(".river")
Out[3]:
[<Selector xpath="descendant-or-self::*[@class and contains(concat(' ', normalize-space(@class), ' '), ' fleuve ')]" data='<div class="fleuve">\n   <section>\n      '>,
<Selector xpath="descendant-or-self::*[@class and contains(concat(' ', normalize-space(@class), ' '), ' fleuve ')]" data='<div class="fleuve">\n</div>'>]
```

Pour récupérer chacun des articles, il faut adresser les balises
`<article>` contenues dans le sélecteur:

```
In [4]: response.css(".river")[0].css(".teaser")
Out[4]:
[<Selector xpath='descendant-or-self::article' data='<article class="grid_12 alpha enrichi mg'>,
<Selector xpath='descendant-or-self::article' data='<article class="grid_12 alpha enrichi">\n'>,
<Selector xpath='descendant-or-self::article' data='<article class="grid_12 alpha enrichi">\n'>,
<Selector xpath='descendant-or-self::article' data='<article class="grid_12 alpha enrichi">\n'>,
<Selector xpath='descendant-or-self::article' data='<article class="grid_12 alpha enrichi">\n'>,
<Selector xpath='descendant-or-self::article' data='<article class="grid_12 alpha enrichi">\n'>,
<Selector xpath='descendant-or-self::article' data='<article class="grid_12 alpha enrichi">\n'>,
<Selector xpath='descendant-or-self::article' data='<article class="grid_12 alpha enrichi">\n'>]   
```

Comme précédemment, on peut empiler les sélecteurs `css` pour créer des
requêtes plus complexes.

Par exemple, pour récupérer tous les titres des différents articles :

```
In [8]: response.css(".river")[0].css(".teaser h3::text").extract()
Out[8]:
['«\xa0Le Livre noir de Gaza\xa0»\xa0: décryptage d’une entreprise d’anéantissement',
 'Au Maroc, la rue se mobilise pour Gaza, le pouvoir maintient sa coopération avec Israël',
 'La guerre à Gaza ravive de manière inédite le traumatisme des Palestiniens',
 '7-Octobre\xa0: Benyamin Nétanyahou sous la menace de la justice internationale',
 'En France, un mouvement propalestinien phagocyté par LFI',
 'En direct, guerre au\xa0Proche-Orient\xa0: le Hezbollah dit soutenir les efforts du Liban en vue d’un cessez-le-feu',
 'A Gaza, des destructions agricoles sans précédent qui accroissent le risque de famine',
 'La Chine veut croire à la reprise de son économie',
 'Comment «\xa0Le Monde\xa0» a couvert un an de guerre à Gaza',
 'Guerre Israël-Hamas\xa0: «\xa0L’enjeu consiste à admettre que le salut ne vient jamais de la puissance, uniquement de la négociation\xa0»',
 'En cartes\xa0: un an de destructions dans la bande de Gaza', 
...]
```

En HTML les données sont souvent de très mauvaise qualité. Il faut
définir des méthodes permettant de les nettoyer pour être intégrées dans
des bases de données.

Par exemple, pour supprimer tous les espaces superflus :

In [12]:
def clean_spaces(string_):
    if string_ is not None:
        return " ".join(string_.split())

Pour l'appliquer à tous les titres récupérés, on peut faire une list
comprehension : 
```
In [11]: [clean_spaces(article) for article in response.css(".river")[0].css(".teaser h3::text").extract()]  

Out[11]: ['« Le Livre noir de Gaza » : décryptage d’une entreprise d’anéantissement',
 'Au Maroc, la rue se mobilise pour Gaza, le pouvoir maintient sa coopération avec Israël',
 'La guerre à Gaza ravive de manière inédite le traumatisme des Palestiniens',
 '7-Octobre : Benyamin Nétanyahou sous la menace de la justice internationale',
 'En France, un mouvement propalestinien phagocyté par LFI',
 'En direct, guerre au Proche-Orient : le Hezbollah dit soutenir les efforts du Liban en vue d’un cessez-le-feu',
 'A Gaza, des destructions agricoles sans précédent qui accroissent le risque de famine',
 'La Chine veut croire à la reprise de son économie',
...
]

```

La méthode précédente est intéressante si l'on ne recherche qu'une seule
information par article.

Par contre si l'on veut récupérer d'autres caractéristiques comme
l'image ou la description par exemple, il est plus intéressant et plus
efficace de récupérer l'objet et d'effectuer plusieurs traitements sur
ce dernier.

Chaque objet retourné par les requêtes `css` est un selecteur avec
lequel on peut interagir.

Par exemple pour récupérer le titre, l'image et la description :

```
In [19]: for article in response.css(".river")[0].css(".teaser"):
    ...:     title = clean_spaces(article.css('h3::text').extract_first())
    ...:     image = article.css('img::attr(data-src)').extract_first()
    ...:     description = article.css('p::text').extract_first()
    ...:     print(f"Title: {title} \nImage: {image} \nDescription: {description}\n-----")
```

```
Title: Le Nobel de médecine honore deux pionniers de la régulation des gènes par de tout petits ARN 
Image: https://img.lemde.fr/2024/10/07/35/0/3000/2000/110/73/60/0/6dafcf3_1728304381125-063-2176426487.jpg 
Description: Les deux lauréats américains, Victor Ambros et Gary Ruvkun, sont récompensés pour l’identification d’une voie de contrôle de l’activité des gènes par des micro-ARN, qui est présente dans toutes les cellules animales et végétales.
-----
Title: La position de La France insoumise sur le 7-Octobre, cause de fractures à gauche 
Image: https://img.lemde.fr/2024/10/05/0/0/5304/3536/110/73/60/0/9c38cc7_1728124287163-000-34l44a7.jpg 
Description: « 7-Octobre, un an après ». Alors que Jean-Luc Mélenchon et ses troupes ont érigé la cause palestinienne comme un axe de campagne électorale, les socialistes ont condamné le refus des « insoumis » de qualifier le Hamas de mouvement « terroriste ».
-----
```


## Persistence des données

Pour pouvoir stocker les informations que l'on récupère en parcourant un
site, il faut pouvoir les stocker. On utilise soit de simples
dictionnaires Python, ou mieux des `scrapy.Item` qui sont des
dictionnaires améliorés.

Nous allons voir les deux façons de faire. On peut réécrire la méthode
`parse_category()` pour lui faire retourner un dictionnaire
correspondant à chaque article rencontré :


In [13]:
def parse_category(self, response):
    for article in response.css(".river")[0].css(".teaser"):
        title = self.clean_spaces(article.css("h3::text").extract_first())
        image = article.css("img::attr(data-src)").extract_first()
        description = article.css("p::text").extract_first()
        yield {
            "title": title,
            "image": image,
            "description": description
        }

Si on combine tout dans la spider :

In [14]:
# %load newscrawler/newscrawler/spiders/lemonde_v3.py
import scrapy
from scrapy import Request


class LemondeSpider(scrapy.Spider):
    name = "lemondev3"
    allowed_domains = ["www.lemonde.fr"]
    start_urls = ['https://www.lemonde.fr']

    def parse(self, response):
        title = response.css('title::text').extract_first()
        all_links = {
            name: response.urljoin(url) for name, url in zip(
                response.css("#nav-markup .Nav__item")[3].css("a::text").extract(),
                response.css("#nav-markup .Nav__item")[3].css("a::attr(href)").extract())
        }
        for link in all_links.values():
            yield Request(link, callback=self.parse_category)

    def parse_category(self, response):
        for article in response.css(".river")[0].css(".teaser"):
            title = self.clean_spaces(article.css("h3::text").extract_first())
            image = article.css("img::attr(data-src)").extract_first()
            description = article.css(".txt3::text").extract_first()
            yield {
                "title": title,
                "image": image,
                "description": description
            }

    def clean_spaces(self, string):
        if string:
            return " ".join(string.split())

On peut maintenant lancer notre spider avec la commande suivante :
```
scrapy crawl <SPIDER_NAME>
```

`scrapy crawl` permet de démarrer le processus en allant chercher la
classe `scrapy.Spider` dont l'attribut `name` = &lt;NAME&gt;.

Par exemple, pour la spider `LeMondeSpider` :

In [15]:
!cd newscrawler && scrapy crawl lemondev3

2024-11-26 15:20:47 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: newscrawler)
2024-11-26 15:20:47 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.10.0, Python 3.12.5 (tags/v3.12.5:ff3bc82, Aug  6 2024, 20:45:27) [MSC v.1940 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Windows-11-10.0.26100-SP0
2024-11-26 15:20:47 [scrapy.addons] INFO: Enabled addons:
[]
2024-11-26 15:20:47 [py.warnings] WARNING: C:\Users\nolan\.virtualenvs\DataEngineerTools-r2yFN37n\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrap

On peut exporter les résultats de ces retours dans différents formats de
fichiers.

-   CSV : `scrapy crawl lemondev3 -o lemonde.csv`
-   JSON : `scrapy crawl lemondev3 -o lemonde.json`
-   JSONLINE : `scrapy crawl lemondev3 -o lemonde.jl`
-   XML : `scrapy crawl lemondev3 -o lemonde.xml`

### Exercice :

Exécuter la spider avec les différents formats de stockage.
Explorer ensuite le contenu des fichiers ainsi créés.

## Votre premier Item

La classe `Item` permet de structurer les données que l'on souhaite
récupérer sous la forme d'un modèle. Les items doivent être définis dans
le fichier `items.py` créé par la commande `scrapy startproject`. Les
`Item` héritent de la class `scrapy.Item`.

On veut structurer les données avec trois champs : le titre, l'image et la description. Scrapy utilise une classe `scrapy.Field` permettant de 'déclarer' ces champs. Dans notre cas :

In [16]:
# %load newscrawler/newscrawler/items.py

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class ArticleItem(scrapy.Item):
    title = scrapy.Field()
    image = scrapy.Field()
    description = scrapy.Field()

Utiliser la classe `scrapy.Item` plutôt qu'un simple dictionnaire permet
plus de contrôle sur la structure des données. En effet, on ne peut
insérer dans les items que des données avec des clés 'déclarées'. Ce qui
assure une plus grande cohérence au sein d'un projet.

On peut instancier un item de plusieurs façons :

In [17]:
article_item = ArticleItem(title="Yann Lecun and Meta sortent Llama 3.2", image=None, description="LLama 3.2 n'est pas disponible en france")

In [18]:
print(article_item)

{'description': "LLama 3.2 n'est pas disponible en france",
 'image': None,
 'title': 'Yann Lecun and Meta sortent Llama 3.2'}


In [19]:
article_item = ArticleItem()
article_item["title"] = 'Léon Marchand recoit la légion d\'honneur'
article_item["description"] = 'Quadruple champion olympic'

In [20]:
print(article_item)

{'description': 'Quadruple champion olympic',
 'title': "Léon Marchand recoit la légion d'honneur"}


La définition d'un item permet de palier toutes les erreurs de typo dans
les champs.

In [21]:
article_item = ArticleItem()
article_item["titelkjwnxvmnscbvmknxc"] = 'Erreur exemple'

KeyError: 'ArticleItem does not support field: titelkjwnxvmnscbvmknxc'

Les items héritent des dictionnaires Python, et possèdent donc toutes
les méthodes de ceux-ci :

In [22]:
article_item = ArticleItem(title="Nouvelle lamborghini")
print(article_item["title"])  # Méthode __getitem__()
print(article_item.get("description", "no description provided"))  # Méthode get()

Nouvelle lamborghini
no description provided


On peut transformer un `Item` en dictionnaire très facilement, en le
passant au constructeur:

In [23]:
article_item = ArticleItem(title="Drone DJI")
print(type(article_item))
dict_item = dict(article_item)
print(type(dict_item))
print(dict_item)

<class '__main__.ArticleItem'>
<class 'dict'>
{'title': 'Drone DJI'}


On intègre maintenant cet item dans notre spider.

In [24]:
# %load newscrawler/newscrawler/spiders/lemonde_v4.py

class LemondeSpider(scrapy.Spider):
    name = "lemondev4"
    allowed_domains = ["www.lemonde.fr"]
    start_urls = ['https://www.lemonde.fr']

    def parse(self, response, **kwargs):
        all_links = {
            name: response.urljoin(url) for name, url in zip(
                response.css("#nav-markup .Nav__item")[3].css("a::text").extract(),
                response.css("#nav-markup .Nav__item")[3].css("a::attr(href)").extract())
        }
        for link in all_links.values():
            yield Request(link, callback=self.parse_category)

    def parse_category(self, response):
        for article in response.css(".river")[0].css(".teaser"):
            title = self.clean_spaces(article.css("h3::text").extract_first())
            image = article.css("img::attr(data-src)").extract_first()
            description = article.css("p::text").extract_first()
            yield ArticleItem(
                title=title,
                image=image,
                description=description
            )

    def clean_spaces(self, string):
        if string:
            return " ".join(string.split())

On voit bien que le générateur retourne maintenant un `Item`.

### Exercice : 

Relancer la spider pour vérifier le bon déroulement de l'extraction.


In [25]:
!cd newscrawler && scrapy crawl lemondev4

2024-11-26 15:21:27 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: newscrawler)
2024-11-26 15:21:27 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.10.0, Python 3.12.5 (tags/v3.12.5:ff3bc82, Aug  6 2024, 20:45:27) [MSC v.1940 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Windows-11-10.0.26100-SP0
2024-11-26 15:21:27 [scrapy.addons] INFO: Enabled addons:
[]
2024-11-26 15:21:27 [py.warnings] WARNING: C:\Users\nolan\.virtualenvs\DataEngineerTools-r2yFN37n\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrap

## Postprocessing

Si l'on se réfère au diagramme d'architecture de Scrapy, on voit qu'il
est possible d'insérer des composants supplémentaires dans le flux de
traitement. Ces composants s'appellent `Pipelines`.

Par défaut, tous les `Item` générés au sein d'un projet Scrapy passent
par les `Pipelines`. Les pipelines sont utilisés la plupart du temps
pour :

-   Nettoyer du contenu HTML ;
-   Valider les données scrapées ;
-   Supprimer les items qu'on ne souhaite pas stocker ;
-   Stocker ces objets dans des bases de données.

Les pipelines doivent être définis dans le fichier `pipelines.py`.

Dans notre cas, on peut vouloir nettoyer le champ `title` pour enlever
les caractères superflus.

Nous allons alors transferer la fonction de nettoyage du code html dans
une Pipeline.

In [26]:
# %load newscrawler/newscrawler/pipelines.py

# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html

from scrapy.exceptions import DropItem


class TextPipeline(object):

    def process_item(self, item, spider):
        if item['title']:
            item["title"] = clean_spaces(item["title"])
            return item
        else:
            raise DropItem(f"Missing title in {item}")


def clean_spaces(string):
    if string:
        return " ".join(string.split())

Pour dire au process Scrapy de faire transiter les items par ces
pipelines. Il faut le spécifier dans le fichier de paramétrage
`settings.py`.

In [27]:
ITEM_PIPELINES = {
    'newscrawler.pipelines.TextPipeline': 300,
}

On peut maintenant supprimer la fonction `clean_spaces()` de
l'extraction des données et laisser le Pipeline faire son travail. La
valeur entière définie permet de déterminer l'ordre dans lequel les
pipelines vont être appelés. Ces entiers peuvent être compris entre 0 et
1000.

On relance notre spider :
```
scrapy crawl lemondev4 -o ../data/articles.json
```

On peut aussi utiliser les Pipelines pour stocker les données récupérées
dans une base de données.

La prochaine partie du cours explique plus en détail le fonctionnement de Mongo, mais voici une bref introduction

Pour stocker les items dans des documents mongo :

In [28]:
import pymongo


class MongoPipeline(object):
    collection_name = 'scrapy_items'

    def open_spider(self, spider):
        self.client = pymongo.MongoClient()
        self.db = self.client["lemonde"]

    def close_spider(self, spider):
        self.client.close()

    def process_item(self, item, spider):
        self.db[self.collection_name].insert_one(dict(item))
        return item

Ici on redéfinit deux autres méthodes: `open_spider()`et
`close_spider()`, ces méthodes sont appelées comme leur nom l'indique,
lorsque la Spider est instanciée et fermée.

Ces méthodes nous permettent d'ouvrir la connexion Mongo et de la fermer
lorsque le scraping se termine. La méthode `process_item()` est appelé à
chaque fois qu'un item passe dans le mécanisme interne de scrapy. Ici,
la méthode permet d'insérer l'item en tant que document mongo.

Pour que ce pipeline soit appelé il faut l'ajouter dans les settings du
projet.

In [29]:
ITEM_PIPELINES = {
    'newscrawler.pipelines.TextPipeline': 100,
    'newscrawler.pipelines.MongoPipeline': 300
}

Le pipeline est ajoutée à la fin du process pour profiter des deux
précédents.

## Settings

Scrapy permet de gérer le comportement des spiders avec certains
paramètres. Comme expliqué dans le premier cours, il est important de
suivre des règles en respectant la structure des différents sites. Il
existe énormément de paramètres mais nous allons (dans le cadre de ce
cours) aborder les plus utilisés :

-   DOWNLOAD\_DELAY : Le temps de téléchargement entre chaque requête
    sur le même domaine ;
-   CONCURRENT\_REQUESTS\_PER\_DOMAIN : Nombre de requêtes simultanées
    par domaine ;
-   CONCURRENT\_REQUESTS\_PER\_IP : Nombre de requêtes simultanées par
    IP ;
-   DEFAULT\_REQUEST\_HEADERS : Headers HTTP utilisés pour les requêtes
    ;
-   ROBOTSTXT\_OBEY : Scrapy récupère le robots.txt et adapte le
    scraping en fonction des règles trouvées ;
-   USER\_AGENT : UserAgent utilisé pour faire les requêtes ;
-   BOT\_NAME : Nom du bot annoncé lors des requêtes
-   HTTPCACHE\_ENABLED : Utilisation du cache HTTP, utile lors du
    parcours multiple de la même page.

Le fichiers `settings.py` permet de définir les paramètres globaux d'un
projet. Si votre projet contient un grand nombre de spiders, il peut
être intéressant d'avoir des paramètres distincts pour chaque spider. Un
moyen simple est d'ajouter un attribut `custom_settings` à votre spider
:

In [30]:
class LeMondeSpider(scrapy.Spider):
    name = "lemonde"
    allowed_domains = ["lemonde.fr"]
    start_urls = ['http://lemonde.fr/']
    custom_settings = {
        "HTTPCACHE_ENABLED": True,
        "CONCURRENT_REQUESTS_PER_DOMAIN": 100
    }

C'est la fin de l'introduction à Scrapy. Vous pouvez passer à la suite `3Mongo`